In [8]:
#### use specified quality flag (QA_valid) as the criteria
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os, sys, time
import gdal
import datetime
import fnmatch
import math
import pandas as pd
from glob import glob
import warnings
warnings.simplefilter(action='ignore')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [9]:
year = 2016
sif_file = 'sif/mcd43_b7_sza_vpd_temp_sif_training.nc4'
avhrr_path = 'www.ncei.noaa.gov/data/avhrr-land-surface-reflectance/access'
avhrr_path = avhrr_path + '/' + str(year) + '/'
prefix ='AVHRR-Land_v005_AVH09C1_NOAA-19_'

In [10]:
yaos_sif = xr.open_dataset(sif_file)
sif = yaos_sif.copy()

df = pd.DataFrame(sif.yday.values, columns = ['yday']) 
df['lat'] = sif.lat.values.round(3)
df['lon'] = sif.lon.values.round(3)
df['sif'] = sif.sif_757nm.values
df['b1']  = sif.b1.values
df['b2']  = sif.b2.values
df['sza'] = sif.sza.values

df['doy']  = sif.yday.values - year*1000
df['doy']  = df['doy'] - 1 
df['date'] = pd.to_datetime(df.doy,unit='D',origin=str(year))
#df['month'] = pd.DatetimeIndex(df['date']).month
df['year'] = pd.DatetimeIndex(df['date']).year
#df['day'] = pd.DatetimeIndex(df['date']).day
# df[df.yday == 2015365]

df['lon'] = df['lon'].round(decimals=3)
df['lat'] = df['lat'].round(decimals=3)

In [11]:
df_new = df.where(df.year == year)
df_new = df_new.dropna(axis=0)
df_new['lon'] = df_new['lon'].round(decimals=3)
df_new['lat'] = df_new['lat'].round(decimals=3)

df_new = df_new.drop(columns=['doy','year'])
final_df = df_new.copy()

In [5]:
#year = 2016
#i= 201 # df_new.doy = 13
#date = datetime.datetime(year, 1, 1) + datetime.timedelta(i - 1)
#date
value = [2,4,5,6,7,10,11,12,13,14]
value = [2**x for x in value]
#print(len(value),value)

[4, 16, 32, 64, 128, 1024, 2048, 4096, 8192, 16384]

In [6]:
final = []
for i in range(2):
    for j in range(2):
        for k in range(2):
            for m in range(2):
                for n in range(2):
                    for o in range(2):
                        for p in range(2):
                            for q in range(2):
                                for s in range(2):
                                    for t in range(2):
                                        number = list([i,j,k,m,n,o,p,q,s,t])
                                        final_number = value[0]*i + value[1]*j + value[2]*k + value[3]*m + value[4]*n + \
                                                       value[5]*o + value[6]*p + value[7]*q + value[8]*s + value[9]*t
                                        final.append(final_number)
                                        #print(number, final_number)
# print(sorted(final))
QA_valid = np.unique(final)
len(QA_valid)

1024

In [6]:
i = 1
date = datetime.datetime(year, 1, 1) + datetime.timedelta(i - 1)
avhrr_file = avhrr_path + prefix + format(int(year), '04d') + format(int(date.month), '02d') + format(int(date.day), '02d') + '_*.nc'
avhrr = xr.open_dataset(glob(avhrr_file)[0])

In [8]:
i = 1
final_df = df_new.copy()
date = datetime.datetime(year, 1, 1) + datetime.timedelta(i - 1)
avhrr_file = avhrr_path + prefix + format(int(year), '04d') + format(int(date.month), '02d') + format(int(date.day), '02d') + '_*.nc'
avhrr = xr.open_dataset(glob(avhrr_file)[0])
avhrr = avhrr.sel(latitude = slice(45,-40))

avhrr_red = pd.DataFrame(avhrr.SREFL_CH1.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
avhrr_red.index.name = 'lat'
avhrr_red.columns.name = 'lon'
avhrr_red = avhrr_red.stack(dropna=True)
avhrr_red.name = 'red'
avhrr_red = avhrr_red.reset_index()
avhrr_red['date'] = date
avhrr_red['lon'] = avhrr_red['lon'].round(3)
avhrr_red['lat'] = avhrr_red['lat'].round(3)

avhrr_nir = pd.DataFrame(avhrr.SREFL_CH2.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
avhrr_nir.index.name = 'lat'
avhrr_nir.columns.name = 'lon'
avhrr_nir = avhrr_nir.stack(dropna=True)
avhrr_nir.name = 'nir'
avhrr_nir = avhrr_nir.reset_index()
avhrr_nir['lon'] = avhrr_nir['lon'].round(3)
avhrr_nir['lat'] = avhrr_nir['lat'].round(3)


avhrr_bt3 = pd.DataFrame(avhrr.BT_CH3.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
avhrr_bt3.index.name = 'lat'
avhrr_bt3.columns.name = 'lon'
avhrr_bt3 = avhrr_bt3.stack(dropna=True)
avhrr_bt3.name = 'bt3'
avhrr_bt3 = avhrr_bt3.reset_index()
avhrr_bt3['lon'] = avhrr_bt3['lon'].round(3)
avhrr_bt3['lat'] = avhrr_bt3['lat'].round(3)

avhrr_bt4 = pd.DataFrame(avhrr.BT_CH4.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
avhrr_bt4.index.name = 'lat'
avhrr_bt4.columns.name = 'lon'
avhrr_bt4 = avhrr_bt4.stack(dropna=True)
avhrr_bt4.name = 'bt4'
avhrr_bt4 = avhrr_bt4.reset_index()
avhrr_bt4['lon'] = avhrr_bt4['lon'].round(3)
avhrr_bt4['lat'] = avhrr_bt4['lat'].round(3)

avhrr_bt5 = pd.DataFrame(avhrr.BT_CH5.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
avhrr_bt5.index.name = 'lat'
avhrr_bt5.columns.name = 'lon'
avhrr_bt5 = avhrr_bt5.stack(dropna=True)
avhrr_bt5.name = 'bt5'
avhrr_bt5 = avhrr_bt5.reset_index()
avhrr_bt5['lon'] = avhrr_bt5['lon'].round(3)
avhrr_bt5['lat'] = avhrr_bt5['lat'].round(3)


avhrr_szen = pd.DataFrame(avhrr.SZEN.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
avhrr_szen.index.name = 'lat'
avhrr_szen.columns.name = 'lon'
avhrr_szen = avhrr_szen.stack(dropna=True)
avhrr_szen.name = 'szen'
avhrr_szen = avhrr_szen.reset_index()
avhrr_szen['lon'] = avhrr_szen['lon'].round(3)
avhrr_szen['lat'] = avhrr_szen['lat'].round(3)

avhrr_qa = pd.DataFrame(avhrr.QA.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
avhrr_qa.index.name = 'lat'
avhrr_qa.columns.name = 'lon'
avhrr_qa = avhrr_qa.stack(dropna=True)
avhrr_qa.name = 'QA'
avhrr_qa = avhrr_qa.reset_index()
avhrr_qa['lon'] = avhrr_qa['lon'].round(3)
avhrr_qa['lat'] = avhrr_qa['lat'].round(3)

avhrr_df       = pd.concat([avhrr_red,avhrr_nir.nir,avhrr_bt3.bt3, avhrr_bt4.bt4,avhrr_bt5.bt5,avhrr_szen.szen,avhrr_qa.QA],axis=1,join='inner')
final_df_day   = final_df[final_df.date==date]
final_df_merge = pd.merge(avhrr_df, final_df_day, on=['lat','lon','date'])

#df1['e'] = pd.Series(np.random.randn(sLength), index=df1.index)
final_df_merge.to_csv('out_2016_IR.csv', index=False)

# mark = avhrr_qa.QA.isin(QA_valid)

# avhrr_qa1   = avhrr_qa[mark]
# avhrr_szen1 = avhrr_szen[mark]
# avhrr_nir1  = avhrr_nir[mark]
# avhrr_red1  = avhrr_red[mark]

# avhrr_df       = pd.concat([avhrr_red1,avhrr_nir1.nir,avhrr_szen1.szen,avhrr_qa1.QA],axis=1,join='inner')
# final_df_day   = final_df[final_df.date==date]
# final_df_merge = pd.merge(avhrr_df, final_df_day, on=['lat','lon','date'])

# final_df_merge.to_csv('out_2016_QA_more.csv', index=False)

In [10]:
tmp_df = df_new.copy()

for i in range(2,367):
    print(i)
    # i= 14 # df_new.doy = 13
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(i - 1)
    avhrr_file = avhrr_path + prefix + format(int(year), '04d') + format(int(date.month), '02d') + format(int(date.day), '02d') + '_*.nc'
    #print(avhrr_file)
    avhrr = xr.open_dataset(glob(avhrr_file)[0])
    avhrr = avhrr.sel(latitude = slice(45,-40))
    
    print('Reading AVHRR red, nir, szen, QA')
    avhrr_red = pd.DataFrame(avhrr.SREFL_CH1.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_red.index.name = 'lat'
    avhrr_red.columns.name = 'lon'
    avhrr_red = avhrr_red.stack(dropna=True)
    avhrr_red.name = 'red'
    avhrr_red = avhrr_red.reset_index()
    avhrr_red['date'] = date
    avhrr_red['lon'] = avhrr_red['lon'].round(3)
    avhrr_red['lat'] = avhrr_red['lat'].round(3)
    
#     print('Reading AVHRR nir band')
    avhrr_nir = pd.DataFrame(avhrr.SREFL_CH2.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_nir.index.name = 'lat'
    avhrr_nir.columns.name = 'lon'
    avhrr_nir = avhrr_nir.stack(dropna=True)
    avhrr_nir.name = 'nir'
    avhrr_nir = avhrr_nir.reset_index()
    avhrr_nir['lon'] = avhrr_nir['lon'].round(3)
    avhrr_nir['lat'] = avhrr_nir['lat'].round(3)


    avhrr_bt3 = pd.DataFrame(avhrr.BT_CH3.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_bt3.index.name = 'lat'
    avhrr_bt3.columns.name = 'lon'
    avhrr_bt3 = avhrr_bt3.stack(dropna=True)
    avhrr_bt3.name = 'bt3'
    avhrr_bt3 = avhrr_bt3.reset_index()
    avhrr_bt3['lon'] = avhrr_bt3['lon'].round(3)
    avhrr_bt3['lat'] = avhrr_bt3['lat'].round(3)

    avhrr_bt4 = pd.DataFrame(avhrr.BT_CH4.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_bt4.index.name = 'lat'
    avhrr_bt4.columns.name = 'lon'
    avhrr_bt4 = avhrr_bt4.stack(dropna=True)
    avhrr_bt4.name = 'bt4'
    avhrr_bt4 = avhrr_bt4.reset_index()
    avhrr_bt4['lon'] = avhrr_bt4['lon'].round(3)
    avhrr_bt4['lat'] = avhrr_bt4['lat'].round(3)

    avhrr_bt5 = pd.DataFrame(avhrr.BT_CH5.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_bt5.index.name = 'lat'
    avhrr_bt5.columns.name = 'lon'
    avhrr_bt5 = avhrr_bt5.stack(dropna=True)
    avhrr_bt5.name = 'bt5'
    avhrr_bt5 = avhrr_bt5.reset_index()
    avhrr_bt5['lon'] = avhrr_bt5['lon'].round(3)
    avhrr_bt5['lat'] = avhrr_bt5['lat'].round(3)

#     print('Reading AVHRR szen')
    avhrr_szen = pd.DataFrame(avhrr.SZEN.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_szen.index.name = 'lat'
    avhrr_szen.columns.name = 'lon'
    avhrr_szen = avhrr_szen.stack(dropna=True)
    avhrr_szen.name = 'szen'
    avhrr_szen = avhrr_szen.reset_index()
    avhrr_szen['lon'] = avhrr_szen['lon'].round(3)
    avhrr_szen['lat'] = avhrr_szen['lat'].round(3)

#     print('Reading AVHRR QA')
    avhrr_qa = pd.DataFrame(avhrr.QA.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_qa.index.name = 'lat'
    avhrr_qa.columns.name = 'lon'
    avhrr_qa = avhrr_qa.stack(dropna=True)
    avhrr_qa.name = 'QA'
    avhrr_qa = avhrr_qa.reset_index()
    avhrr_qa['lon'] = avhrr_qa['lon'].round(3)
    avhrr_qa['lat'] = avhrr_qa['lat'].round(3)
    
    print('Finished reading. Here no QA filtering.')    
    avhrr_df     = pd.concat([avhrr_red,avhrr_nir.nir,avhrr_bt3.bt3, avhrr_bt4.bt4,avhrr_bt5.bt5,avhrr_szen.szen,avhrr_qa.QA],axis=1,join='inner')
    print('AVHRR concate finished, the number of datapoints: ', avhrr_df.shape)
    
    tmp_df_day   = tmp_df[tmp_df.date==date]
    tmp_df_merge = pd.merge(avhrr_df, tmp_df_day, on=['lat','lon','date'], sort=False)
    print('AVHRR & MODIS merge finished, the number of datapoints:', tmp_df_merge.shape)
    
    final_df_merge = final_df_merge.append(tmp_df_merge, ignore_index=True, sort=False)
    final_df_merge.to_csv('out_2016_IR.csv', index=False)
    print('Saved to CSV file, the number of total datapoints:', final_df_merge.shape)   
    

2
Reading AVHRR red, nir, szen, QA
Finished reading. Here no QA filtering.
AVHRR concate finished, the number of datapoints:  (3321800, 10)
AVHRR & MODIS merge finished, the number of datapoints: (91, 15)
Saved to CSV file, the number of total datapoints: (153, 15)
3
Reading AVHRR red, nir, szen, QA
Finished reading. Here no QA filtering.
AVHRR concate finished, the number of datapoints:  (3320224, 10)
AVHRR & MODIS merge finished, the number of datapoints: (79, 15)
Saved to CSV file, the number of total datapoints: (232, 15)
4
Reading AVHRR red, nir, szen, QA
Finished reading. Here no QA filtering.
AVHRR concate finished, the number of datapoints:  (3322801, 10)
AVHRR & MODIS merge finished, the number of datapoints: (66, 15)
Saved to CSV file, the number of total datapoints: (298, 15)
5
Reading AVHRR red, nir, szen, QA
Finished reading. Here no QA filtering.
AVHRR concate finished, the number of datapoints:  (3321989, 10)
AVHRR & MODIS merge finished, the number of datapoints: (68, 1